# Import Libraries and Data

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option("display.max_rows", 1000)

import sys
sys.path.append('../scripts')

from html_fiction_parser import *

%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv('../output/html_parse_2019-05-01.csv', index_col=[0])

index_to_exclude = [i for i in range(1,63)]
df = df.drop(index_to_exclude, axis=0)
df = df.reset_index()
df.shape

(2119, 9)

In [3]:
df.reset_index(inplace=True)
df['para_index'] = df['level_0']
df.head(10)


,level_0,index,para_index,tag,para,num_quotes_up_to_para,num_utterances,raw_utter_list,tokenized_sent,tokenized_sent_clean
0,0,0,0,narrative,By Jane Austen,0,0,[],[N],[N]
1,1,63,1,narrative,Chapter 1,0,0,[],[N],[N]
2,2,64,2,narrative,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.",0,0,[],[N],[N]
3,3,65,3,narrative,"However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.",0,0,[],[N],[N]
4,4,66,4,utterance,"""My dear Mr. Bennet,"" said his lady to him one day, ""have you heard that Netherfield Park is let at last?""",4,2,"['""My dear Mr. Bennet,""', '""have you heard that Netherfield Park is let at last?""']","""My dear Mr Bennet,"" [N] ""have you heard that Netherfield Park is let at last?""","My dear Mr Bennet, [N] have you heard that Netherfield Park is let at last?"
5,5,67,5,narrative,Mr. Bennet replied that he had not.,4,0,[],[N],[N]
6,6,68,6,utterance,"""But it is,"" returned she; ""for Mrs. Long has just been here, and she told me all about it.""",8,2,"['""But it is,""', '""for Mrs. Long has just been here, and she told me all about it.""']","""But it is,"" [N] ""for Mrs Long has just been here, and she told me all about it.""","But it is, [N] for Mrs Long has just been here, and she told me all about it."
7,7,69,7,narrative,Mr. Bennet made no answer.,8,0,[],[N],[N]
8,8,70,8,utterance,"""Do you not want to know who has taken it?"" cried his wife impatiently.",10,1,"['""Do you not want to know who has taken it?""']","""Do you not want to know who has taken it?"" [N]",Do you not want to know who has taken it? [N]
9,9,71,9,utterance,"""You want to tell me, and I have no objection to hearing it.""",12,1,"['""You want to tell me, and I have no objection to hearing it.""']","""You want to tell me, and I have no objection to hearing it.""","You want to tell me, and I have no objection to hearing it."


In [4]:
# df[(df['para'].apply(lambda x: is_quote_start_of_sent(x)==False)) & (df['tag']=='utterance')]

In [10]:
def find_utterances_raw(para):
    quotes = re.findall('"([^"]*)"', para)
    return ['"' + i + '"' for i in quotes]

def remove_period_from_honorifics(para):
    regex = '(Miss.|Mrs.|Mr.|Ms.)'
    for i in re.findall(regex, para, flags=re.IGNORECASE):
        para = para.replace(i, i[:-1])
    return para

def replace_narrative_w_mark(sent_list):
    new_sent_list = []
    for i, sent in enumerate(sent_list):
        if i == 0:
            new_sent_list.append(sent) if is_utterance(sent) else new_sent_list.append(sent)
        elif i == len(sent_list)-1:
            new_sent_list.append(sent) if is_utterance(sent) else new_sent_list.append(sent)
        else:
            if is_utterance(sent) == False:
                if (is_utterance(sent_list[i-1]) & is_utterance(sent_list[i+1])):
                    continue
#                     new_sent_list.append('_')
                else:
                    new_sent_list.append(sent)
            else:
                new_sent_list.append(sent)
    return new_sent_list

def tokenize_para(para):
    para = remove_period_from_honorifics(para)
#     regex = '((?![.\s])[^."]*(?:"[^"]*[^".]"[^."]*)*(?:"[^"]+\."|\.))'
#     regex = '((?![.\s])[^."]*(?:"[^"]*[^".]"[^."]*)*(?:"[^"]+\."|\.|"[^"]+\?"|"[^"]+\!"))'
    regex = r'(("[^"]*")|([^"]*))'
    results = [i[0] for i in re.findall(regex, para)]
    tag_texts = [i for i in results if i]
    output_string = replace_narrative_w_mark(tag_texts)
    return output_string


def tokenize_para_b(para):
    para = remove_period_from_honorifics(para)
#     regex = '((?![.\s])[^."]*(?:"[^"]*[^".]"[^."]*)*(?:"[^"]+\."|\.))'
#     regex = '((?![.\s])[^."]*(?:"[^"]*[^".]"[^."]*)*(?:"[^"]+\."|\.|"[^"]+\?"|"[^"]+\!"))'
    regex = r'(("[^"]*")|([^"]*))'
    results = [i[0] for i in re.findall(regex, para)]
    tag_texts = [i for i in results if i]
    output_string = replace_narrative_w_mark_b(tag_texts)
    return output_string


            

In [6]:
teststring0 = '“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?”'
print(teststring0)

“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?”


In [7]:
tokenize_para(teststring0)

''

In [8]:
teststring0 = '"Come here, child," cried her father as she appeared. "I have sent for you on an affair of importance. I understand that Mr. Collins has made you an offer of marriage. Is it true?" Elizabeth replied that it was. "Very well—and this offer of marriage you have refused?"'
print(teststring0)

"Come here, child," cried her father as she appeared. "I have sent for you on an affair of importance. I understand that Mr. Collins has made you an offer of marriage. Is it true?" Elizabeth replied that it was. "Very well—and this offer of marriage you have refused?"


In [9]:
tokenize_para(teststring0)

'"Come here, child,"  "I have sent for you on an affair of importance. I understand that Mr Collins has made you an offer of marriage. Is it true?"  "Very well—and this offer of marriage you have refused?"'

In [130]:
teststring1 = '"My dear Mr. Bennet," replied his Ms. wife, "how can you be so tiresome! You must know that I am thinking of his marrying one of them."'
print(teststring1)

"My dear Mr. Bennet," replied his Ms. wife, "how can you be so tiresome! You must know that I am thinking of his marrying one of them."


In [131]:
tokenize_para(teststring1)

'"My dear Mr Bennet," [N] "how can you be so tiresome! You must know that I am thinking of his marrying one of them."'

In [19]:
find_utterances_raw(teststring1)

['"My dear Mr. Bennet,"',
 '"how can you be so tiresome! You must know that I am thinking of his marrying one of them."']

In [20]:
teststring2 = '"My dear Mr. Bennet," said his lady to him one day, "have you heard that Netherfield Park is let at last?"'
print(teststring2)

"My dear Mr. Bennet," said his lady to him one day, "have you heard that Netherfield Park is let at last?"


In [21]:
tokenize_para(teststring2)

'"My dear Mr Bennet," [N] "have you heard that Netherfield Park is let at last?"'

In [22]:
df['raw_utter_list'] = df['para'].apply(lambda x: find_utterances_raw(x))

In [23]:
df['tokenized_sent'] = df['para'].apply(lambda x: tokenize_para(x))

In [24]:
df.shape

(2181, 7)

# Cleaning Manual Labeled CSV

In [11]:
df_labeled = pd.read_csv('../../data/parsed-n-labeled-data/pnp-gutenberg-label-task-pride-and-prejudice-by-jane-austen.csv', index_col=[0])

In [12]:
df_labeled = df_labeled.reset_index()
df_labeled.columns = ['para_index', 'tag_old','para', 'position', 'speaker', 'type']

In [13]:
df_labeled['type'] = df_labeled['type'].replace(' c', 'c').replace('c ', 'c').fillna('')

In [14]:
df_labeled[df_labeled['position'].isnull()]

,para_index,tag_old,para,position,speaker,type
344,344,not,He was silent.,NaN,NaN,
1722,1722,not,Chapter 53,NaN,NaN,


In [15]:
df_labeled.loc[344, 'position'] = 'n'
df_labeled.loc[1722, 'position'] = 'n'

In [16]:
df_labeled['position'] = df_labeled['position'].apply(lambda x: x.lower())

In [17]:
df_labeled.loc[343]

para_index    343                                     
tag_old       dialogue-F                              
para          “How can you contrive to write so even?”
position      1                                       
speaker       NaN                                     
type                                                  
Name: 343, dtype: object

In [18]:
df_labeled['position'].value_counts()

n    785
1    689
2    507
3    99 
4    29 
5    8  
6    2  
Name: position, dtype: int64

In [19]:
df_labeled['type'].value_counts()

            1819
c           225 
letter      71  
quote       2   
emphasis    2   
Name: type, dtype: int64

In [20]:
def recount_convo(type_tag, position_tag):
    global convo_count
    
    if (type_tag == 'c') & (position_tag.lower()!='n'):
        convo_count += 1
        return 'c_{}'.format(convo_count)
    elif (type_tag == '') & (position_tag.lower()!='n'):
        return 'c_{}'.format(convo_count)
    elif (type_tag == 'letter'):
        return 'letter'
    elif (type_tag == 'quote'):
        return 'quote'
    else:
        return 'narrative'



In [21]:
convo_count = 0
df_labeled['new_type_tag'] = df_labeled.apply(lambda row: recount_convo(str(row['type']), str(row['position'])), axis=1)

In [22]:
df_labeled.head(10)

,para_index,tag_old,para,position,speaker,type,new_type_tag
0,0,not,By Jane Austen,n,NaN,,narrative
1,1,not,Chapter 1,n,NaN,,narrative
2,2,not,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.",n,NaN,,narrative
3,3,not,"However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.",n,NaN,,narrative
4,4,dialogue-F,"“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?”",1,mother,c,c_1
5,5,not,Mr. Bennet replied that he had not.,n,NaN,,narrative
6,6,dialogue-F,"“But it is,” returned she; “for Mrs. Long has just been here, and she told me all about it.”",1,mother,,c_1
7,7,not,Mr. Bennet made no answer.,n,NaN,,narrative
8,8,dialogue-F,“Do you not want to know who has taken it?” cried his wife impatiently.,1,mother,,c_1
9,9,dialogue-F,"“You want to tell me, and I have no objection to hearing it.”",2,father,,c_1


# Merging

In [23]:
df_merged = pd.merge(df, df_labeled, on='para_index', how='left')

In [24]:
df_merged.shape

(2119, 16)

In [25]:
df_merged.head(10)

,level_0,index,para_index,tag,para_x,num_quotes_up_to_para,num_utterances,raw_utter_list,tokenized_sent,tokenized_sent_clean,tag_old,para_y,position,speaker,type,new_type_tag
0,0,0,0,narrative,By Jane Austen,0,0,[],[N],[N],not,By Jane Austen,n,NaN,,narrative
1,1,63,1,narrative,Chapter 1,0,0,[],[N],[N],not,Chapter 1,n,NaN,,narrative
2,2,64,2,narrative,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.",0,0,[],[N],[N],not,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.",n,NaN,,narrative
3,3,65,3,narrative,"However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.",0,0,[],[N],[N],not,"However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.",n,NaN,,narrative
4,4,66,4,utterance,"""My dear Mr. Bennet,"" said his lady to him one day, ""have you heard that Netherfield Park is let at last?""",4,2,"['""My dear Mr. Bennet,""', '""have you heard that Netherfield Park is let at last?""']","""My dear Mr Bennet,"" [N] ""have you heard that Netherfield Park is let at last?""","My dear Mr Bennet, [N] have you heard that Netherfield Park is let at last?",dialogue-F,"“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?”",1,mother,c,c_1
5,5,67,5,narrative,Mr. Bennet replied that he had not.,4,0,[],[N],[N],not,Mr. Bennet replied that he had not.,n,NaN,,narrative
6,6,68,6,utterance,"""But it is,"" returned she; ""for Mrs. Long has just been here, and she told me all about it.""",8,2,"['""But it is,""', '""for Mrs. Long has just been here, and she told me all about it.""']","""But it is,"" [N] ""for Mrs Long has just been here, and she told me all about it.""","But it is, [N] for Mrs Long has just been here, and she told me all about it.",dialogue-F,"“But it is,” returned she; “for Mrs. Long has just been here, and she told me all about it.”",1,mother,,c_1
7,7,69,7,narrative,Mr. Bennet made no answer.,8,0,[],[N],[N],not,Mr. Bennet made no answer.,n,NaN,,narrative
8,8,70,8,utterance,"""Do you not want to know who has taken it?"" cried his wife impatiently.",10,1,"['""Do you not want to know who has taken it?""']","""Do you not want to know who has taken it?"" [N]",Do you not want to know who has taken it? [N],dialogue-F,“Do you not want to know who has taken it?” cried his wife impatiently.,1,mother,,c_1
9,9,71,9,utterance,"""You want to tell me, and I have no objection to hearing it.""",12,1,"['""You want to tell me, and I have no objection to hearing it.""']","""You want to tell me, and I have no objection to hearing it.""","You want to tell me, and I have no objection to hearing it.",dialogue-F,"“You want to tell me, and I have no objection to hearing it.”",2,father,,c_1


In [26]:
columns_i_want = ['para_index', 'para_x', 'num_utterances', 'raw_utter_list', 'tokenized_sent', 'tokenized_sent_clean', 'position', 'speaker', 'new_type_tag']
df_merged = df_merged[columns_i_want]

In [27]:
df_merged = df_merged.fillna('')

In [28]:
df_merged.columns

Index(['para_index', 'para_x', 'num_utterances', 'raw_utter_list',
       'tokenized_sent', 'tokenized_sent_clean', 'position', 'speaker',
       'new_type_tag'],
      dtype='object')

In [29]:
df_merged['chapter_tag'] = df_merged['para_x'].apply(lambda x: x if 'chapter ' in x.lower() else '')

In [30]:
chapter_dict = dict()
chapter_tag = ''
for i in df_merged.index:
    curr_chapter_tag = df_merged.loc[i]['chapter_tag']
    if curr_chapter_tag == '':
        if chapter_tag != '':
            chapter_dict[i] = chapter_tag
        else:
            chapter_tag = curr_chapter_tag
            chapter_dict[i] = chapter_tag
    else: 
        if chapter_tag == curr_chapter_tag:
            chapter_dict[i] = chapter_tag
        else:
            chapter_tag = curr_chapter_tag
            chapter_dict[i] = chapter_tag



In [31]:
df_merged['new_chapter_tag'] = list(x[1] for x in chapter_dict.items())

In [32]:
def is_utterance(sent):
    return get_num_quotes(sent)%2==0


def tag_tokens(sent_list):
    new_list = []
    for sent in sent_list:
        if is_utterance(sent):
            new_list.append((sent, ))

In [34]:
new_label_dict = dict()
count = 0 
for j in df_merged.index:
    tuple_i_want = (j,) \
                + tuple(list(df_merged.loc[j][['position', 'speaker', 'new_type_tag', 'new_chapter_tag']].values)) \
                + (tokenize_para(df_merged.loc[j]['para_x']),)
    new_label_dict[count] = tuple_i_want
    count += 1     

In [35]:
df_final = pd.DataFrame(list(new_label_dict.items()), columns=['sent_index', 'tag'])

In [36]:
df_final['para_index'] = df_final['tag'].apply(lambda x: x[0])
df_final['position_tag'] = df_final['tag'].apply(lambda x: x[1])
df_final['speaker_tag'] = df_final['tag'].apply(lambda x: x[2])
df_final['type_tag'] = df_final['tag'].apply(lambda x: x[3])
df_final['chapter_tag'] = df_final['tag'].apply(lambda x: x[4])
df_final['tokenized_sent'] = df_final['tag'].apply(lambda x: x[5])


In [37]:
df_final.head(100)

,sent_index,tag,para_index,position_tag,speaker_tag,type_tag,chapter_tag,tokenized_sent
0,0,"(0, n, , narrative, , [By Jane Austen])",0,n,,narrative,,[By Jane Austen]
1,1,"(1, n, , narrative, Chapter 1, [Chapter 1])",1,n,,narrative,Chapter 1,[Chapter 1]
2,2,"(2, n, , narrative, Chapter 1, [It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.])",2,n,,narrative,Chapter 1,"[It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.]"
3,3,"(3, n, , narrative, Chapter 1, [However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.])",3,n,,narrative,Chapter 1,"[However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.]"
4,4,"(4, 1, mother, c_1, Chapter 1, [""My dear Mr Bennet,"", said his lady to him one day, , ""have you heard that Netherfield Park is let at last?""])",4,1,mother,c_1,Chapter 1,"[""My dear Mr Bennet,"", said his lady to him one day, , ""have you heard that Netherfield Park is let at last?""]"
5,5,"(5, n, , narrative, Chapter 1, [Mr Bennet replied that he had not.])",5,n,,narrative,Chapter 1,[Mr Bennet replied that he had not.]
6,6,"(6, 1, mother, c_1, Chapter 1, [""But it is,"", returned she; , ""for Mrs Long has just been here, and she told me all about it.""])",6,1,mother,c_1,Chapter 1,"[""But it is,"", returned she; , ""for Mrs Long has just been here, and she told me all about it.""]"
7,7,"(7, n, , narrative, Chapter 1, [Mr Bennet made no answer.])",7,n,,narrative,Chapter 1,[Mr Bennet made no answer.]
8,8,"(8, 1, mother, c_1, Chapter 1, [""Do you not want to know who has taken it?"", cried his wife impatiently.])",8,1,mother,c_1,Chapter 1,"[""Do you not want to know who has taken it?"", cried his wife impatiently.]"
9,9,"(9, 2, father, c_1, Chapter 1, [""You want to tell me, and I have no objection to hearing it.""])",9,2,father,c_1,Chapter 1,"[""You want to tell me, and I have no objection to hearing it.""]"


In [38]:
index_to_exclude = df_final[df_final['tokenized_sent'].apply(lambda x: x=='[N]')].index

In [39]:
df_final = df_final.drop(index_to_exclude, axis=0)

In [40]:
df_final = df_final.reset_index()
del df_final['index']

In [41]:
df_final.shape

(2119, 8)

In [42]:
label_dict = dict()

conver_tag = ''
position_tag = 'n'

for i in df_final.index:
    curr_position_tag = df_final.loc[i]['position_tag']
    curr_conver_tag = df_final.loc[i]['type_tag']
    if (curr_conver_tag == conver_tag) & (curr_position_tag!='n'):
        if curr_position_tag == position_tag:
            label_dict[i] = 'part'
            position_tag = curr_position_tag
            conver_tag = curr_conver_tag
        else:
            label_dict[i] = 'response'
            position_tag = curr_position_tag
            conver_tag = curr_conver_tag
    else: 
        label_dict[i] = 'not'
        position_tag = curr_position_tag
        conver_tag = curr_conver_tag    

In [43]:
df_final['label'] = list(x[1] for x in label_dict.items())

In [44]:
df_final.head(100)

,sent_index,tag,para_index,position_tag,speaker_tag,type_tag,chapter_tag,tokenized_sent,label
0,0,"(0, n, , narrative, , [By Jane Austen])",0,n,,narrative,,[By Jane Austen],not
1,1,"(1, n, , narrative, Chapter 1, [Chapter 1])",1,n,,narrative,Chapter 1,[Chapter 1],not
2,2,"(2, n, , narrative, Chapter 1, [It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.])",2,n,,narrative,Chapter 1,"[It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.]",not
3,3,"(3, n, , narrative, Chapter 1, [However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.])",3,n,,narrative,Chapter 1,"[However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.]",not
4,4,"(4, 1, mother, c_1, Chapter 1, [""My dear Mr Bennet,"", said his lady to him one day, , ""have you heard that Netherfield Park is let at last?""])",4,1,mother,c_1,Chapter 1,"[""My dear Mr Bennet,"", said his lady to him one day, , ""have you heard that Netherfield Park is let at last?""]",not
5,5,"(5, n, , narrative, Chapter 1, [Mr Bennet replied that he had not.])",5,n,,narrative,Chapter 1,[Mr Bennet replied that he had not.],not
6,6,"(6, 1, mother, c_1, Chapter 1, [""But it is,"", returned she; , ""for Mrs Long has just been here, and she told me all about it.""])",6,1,mother,c_1,Chapter 1,"[""But it is,"", returned she; , ""for Mrs Long has just been here, and she told me all about it.""]",not
7,7,"(7, n, , narrative, Chapter 1, [Mr Bennet made no answer.])",7,n,,narrative,Chapter 1,[Mr Bennet made no answer.],not
8,8,"(8, 1, mother, c_1, Chapter 1, [""Do you not want to know who has taken it?"", cried his wife impatiently.])",8,1,mother,c_1,Chapter 1,"[""Do you not want to know who has taken it?"", cried his wife impatiently.]",not
9,9,"(9, 2, father, c_1, Chapter 1, [""You want to tell me, and I have no objection to hearing it.""])",9,2,father,c_1,Chapter 1,"[""You want to tell me, and I have no objection to hearing it.""]",response


In [45]:
df_final.shape

(2119, 9)

chapters  19–26  asthe  test  set,  chapters  27–33  as  the  developmentset, and the remaining 46 chapters as the trainingset

In [46]:
test_set_chapters = ['Chapter {}'.format(x) for x in range(19,27)]
validation_set_chapters = ['Chapter {}'.format(x) for x in range(27,34)]


In [47]:
df_test = df_final[df_final['chapter_tag'].apply(lambda x: True if x in test_set_chapters else False)]

In [48]:
df_validation = df_final[df_final['chapter_tag'].apply(lambda x: True if x in validation_set_chapters else False)]

In [49]:
def custom_train_test_split(field):
    if field in test_set_chapters:
        return 'test'
    elif field in validation_set_chapters:
        return 'validation'
    else:
        return 'train'

In [50]:
df_final['split_tag'] = df_final['chapter_tag'].apply(lambda x: custom_train_test_split(x))

In [51]:
df_final['split_tag'].value_counts()

train         1700
test          213 
validation    206 
Name: split_tag, dtype: int64

In [52]:
pair_list = []
for i in df_final.index:
    first_utter = df_final.loc[i]['tokenized_sent']
    first_utter_label = df_final.loc[i]['label']
    first_utter_split = df_final.loc[i]['split_tag']
    if i != 0:
        if first_utter_label == 'response':
            prev_utter = df_final.loc[i-1]['tokenized_sent'] 
            pair_list.append((prev_utter, first_utter, 'response', first_utter_split))
        elif first_utter_label == 'part':
            prev_utter = df_final.loc[i-1]['tokenized_sent'] 
            pair_list.append((prev_utter, first_utter, 'part', first_utter_split))

        else:
            prev_utter = df_final.loc[i-1]['tokenized_sent'] 
            pair_list.append((prev_utter, first_utter, 'not_pair', first_utter_split))


In [53]:
pair_list

[(['By Jane Austen'], ['Chapter 1'], 'not_pair', 'train'),
 (['Chapter 1'],
  ['It is a truth universally acknowledged, that a single man in possession of       a good fortune, must be in want of a wife.'],
  'not_pair',
  'train'),
 (['It is a truth universally acknowledged, that a single man in possession of       a good fortune, must be in want of a wife.'],
  ['However little known the feelings or views of such a man may be on his       first entering a neighbourhood, this truth is so well fixed in the minds       of the surrounding families, that he is considered the rightful property       of some one or other of their daughters.'],
  'not_pair',
  'train'),
 (['However little known the feelings or views of such a man may be on his       first entering a neighbourhood, this truth is so well fixed in the minds       of the surrounding families, that he is considered the rightful property       of some one or other of their daughters.'],
  ['"My dear Mr Bennet,"',
   ' said his lad

In [54]:
df_final_pair = pd.DataFrame()

In [63]:
df_final_pair['utter_1'] = [x[0][0] for x in pair_list]
df_final_pair['utter_2'] = [x[1][0] for x in pair_list]
df_final_pair['label'] = [x[2] for x in pair_list]
df_final_pair['split_tag'] = [x[3] for x in pair_list]


In [64]:
df_final_pair.head(100)

,utter_1,utter_2,label,split_tag
0,By Jane Austen,Chapter 1,not_pair,train
1,Chapter 1,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.",not_pair,train
2,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.","However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.",not_pair,train
3,"However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.","""My dear Mr Bennet,""",not_pair,train
4,"""My dear Mr Bennet,""",Mr Bennet replied that he had not.,not_pair,train
5,Mr Bennet replied that he had not.,"""But it is,""",not_pair,train
6,"""But it is,""",Mr Bennet made no answer.,not_pair,train
7,Mr Bennet made no answer.,"""Do you not want to know who has taken it?""",not_pair,train
8,"""Do you not want to know who has taken it?""","""You want to tell me, and I have no objection to hearing it.""",response,train
9,"""You want to tell me, and I have no objection to hearing it.""",This was invitation enough.,not_pair,train


In [65]:
df_final_pair.shape

(2118, 4)

In [58]:
# df_final['tokenized_sent_clean'] = df_final['tokenized_sent'].apply(lambda x: re.sub('"', '', x))

In [59]:
# df_final.head(50)

In [66]:
df_final_pair.to_csv('../../data/parsed-n-labeled-data/labeled-utterance-pair-final-020519-v2.csv')